In [5]:
import random

def csv_to_array(filename):
    file = open(filename, 'r')
    file_text = file.read()
    # Get rid of the first three, which are examples
    split_text = file_text.split('|')[3:]
    i = 0
    tuple_text = []
    while i < len(split_text) - 1:
        tuple_text.append((split_text[i+1], split_text[i], int(i/2)))
        i += 2
    
    return tuple_text
    
def make_training_and_dev_data(all_data):
    random.shuffle(all_data)
    training_ratio = .8
    return all_data[:int(len(all_data) * training_ratio)], all_data[int(len(all_data) * training_ratio):]
#     return all_data[:(len(all_data) * training_ratio)], all_data[:(len(all_data) * training_ratio)] 


official_data = csv_to_array('yelp_data_official_training.csv')

for garbage in official_data:
    if garbage[1].isdigit() is False:
        print(garbage[1])
        break

training_data, dev_data = make_training_and_dev_data(official_data[:1000])
print(training_data[1][0])
print(len(dev_data))
print(training_data[0][2])


"I should've believed what was already written but It was close to my hotel. The WORST mani/pedi of my life. I'm 43. I'm a flight attendant. I've been to a LOT of nail salons. I swear she wasn't licensed. She had no clue but claims to have been doing nails for 40 years. She clearly needs to retire and the male owner was an a$$. He says ""not everyone does it the same."" Well..... They certainly should come close!! No cuticle push back. Didn't know how to remove gel polish. Didn't file straight or smooth. Didn't clean under my nails after lotion. I had to tell her to take alcohol to my nails to clean off lotion and remind her to use a base coat. Then she polished all over my cuticle. 17 hours later having done nothing but sleep workout and shower the polish is chipped off 6 nails."
531
200
914


In [15]:
import nltk
import pandas as pd
import re
import string

from nltk import word_tokenize
from nltk import FreqDist
from nltk.corpus import stopwords


def bag_of_words(data):
    bag_of_words = []
    for tuples in data:
        review = tuples[0]
        token_review = word_tokenize(review)
        bag_of_words += token_review
    
    # Sanitize the bag of words
    stopwords = nltk.corpus.stopwords.words('english')
    # To do: don't use isalpha because we're missing "Dr."
    # REVISIT
    bag_of_words = [w.lower() for w in bag_of_words if w.lower() not in stopwords and w.isalpha() is True]
    return bag_of_words
        
def most_common_words_from_array(word_array, most_common):
    word_fd = FreqDist(word_array)
    most_common_words = word_fd.most_common(most_common)
    sanitized_common_words = [line[0] for line in most_common_words]
    return sanitized_common_words

word_array = bag_of_words(training_data)
most_common_words = most_common_words_from_array(word_array, 1000)
print(most_common_words[:50])

    

['hair', 'would', 'time', 'great', 'one', 'get', 'place', 'like', 'cut', 'back', 'salon', 'go', 'could', 'really', 'good', 'even', 'also', 'going', 'work', 'appointment', 'never', 'always', 'got', 'new', 'service', 'nice', 'know', 'staff', 'done', 'told', 'make', 'first', 'friendly', 'years', 'experience', 'went', 'said', 'best', 'well', 'little', 'job', 'look', 'two', 'take', 'called', 'recommend', 'much', 'pittsburgh', 'made', 'stylist']


/Users/RK/anaconda/lib/python3.5/site-packages/sklearn/utils/fixes.py:64: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  if 'order' in inspect.getargspec(np.copy)[0]:


In [16]:
def contains_words_features(review, words):
    features = {}
    for word in words:
        features["contains_" + str(word)]= (word in review.lower())
        features["contains_"
    return features

print(training_data[0])
contains_words_features(training_data[5][0], most_common_words)


("I have been a patient of Dr. Brown's for many years. He is dedicated to continuing to learn and implement the latest dental techniques.  His office is modern, friendly, and convenient.  His staff is the best office staff I have ever encountered in any type of office anywhere!  They are second to none!\n487", '3', 486)


{'contains_parents': False,
 'contains_customer': False,
 'contains_special': False,
 'contains_fair': False,
 'contains_decent': False,
 'contains_talent': False,
 'contains_pay': False,
 'contains_surgery': False,
 'contains_vet': False,
 'contains_usually': False,
 'contains_want': False,
 'contains_body': False,
 'contains_inside': False,
 'contains_kitchen': False,
 'contains_hours': False,
 'contains_always': False,
 'contains_completely': False,
 'contains_tables': False,
 'contains_stylists': False,
 'contains_teeth': False,
 'contains_light': True,
 'contains_reviews': False,
 'contains_look': False,
 'contains_sort': False,
 'contains_repaired': False,
 'contains_bring': False,
 'contains_properly': True,
 'contains_dry': False,
 'contains_pet': False,
 'contains_change': False,
 'contains_visit': False,
 'contains_particular': False,
 'contains_shelter': False,
 'contains_windows': False,
 'contains_event': True,
 'contains_greeted': False,
 'contains_comfortable': False,
 '

In [17]:
def create_feature_set(naive_set):
    training_set = []
    for item in naive_set:
        feature_dict = contains_words_features(item[0], most_common_words)
        training_set.append((feature_dict, item[1]))
    return training_set



training_features1 = create_feature_set(training_data)
dev_features1 = create_feature_set(dev_data)
cl1 = nltk.NaiveBayesClassifier.train(training_features1)
print ("%.3f" % nltk.classify.accuracy(cl1, dev_features1))



0.885


In [20]:
import re

def test_file_to_array(filename):
    file = open(filename, 'r')
    file_text = file.read()
    # Skip the first one because it's the header starter shit
    split_text = re.split('\d+\|', file_text)[1:]
    tupled_text = []
    for i in range(len(split_text)):
        tupled_text.append((split_text[i], i))        
    return tupled_text

test_array = test_file_to_array('yelp_data_official_test_nocategories.csv')


In [45]:
def run_classifier_on_test_data(test_data, classifier):
    test_feature_sets = create_feature_set(test_data)
    final_goddamn_output = []
    for i in range(len(test_data)):
        final_goddamn_output.append((i, cl1.classify(test_feature_sets[i][0])))
    return final_goddamn_output

def write_output_to_csv(output, filename):
    file = open(filename, 'w')
    for item in output:
        file.write(str(item[0]) + ',' + str(item[1]) + '\n')
    
output = run_classifier_on_test_data(test_array, cl1)
write_output_to_csv(output, 'kuvinka_nicholas_kaggle.csv')
